In [13]:
%matplotlib inline

import os
import zipfile
import urllib

import matplotlib.pyplot as plt

import numpy as np

FOOD_PATH = os.path.join("datasets", "openfoodfacts")
FOOD_TRANSFORMED_PATH_FILE = os.path.join(FOOD_PATH, "fr.openfoodfacts.org.products_transformed.csv")

import seaborn as sns
sns.set()

pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows",1000)


# Import des données

In [10]:
import pandas as pd

def load_food_data(csv_path=FOOD_TRANSFORMED_PATH_FILE):
    return pd.read_csv(csv_path, sep=',', header=0, encoding='utf-8', low_memory=False)



In [11]:
food = load_food_data()

In [14]:
food.head()

,code,last_modified_t,product_name,states_tags,main_category_fr,brands,brands_tags,nutrition_grade_fr,energy_100g,sugars_100g,salt_100g,saturated-fat_100g,fiber_100g,proteins_100g,ingredients_from_palm_oil_n,pnns_groups_2,pnns_groups_1,labels_tags,countries_tags,additives_tags,additives_n,ingredients_text,image_url,nutrition_scoring,no_ingredients,no_ingredients_scoring,additives_nocive_scoring,energy_100g_scoring,salt_100g_scoring,sugars_100g_scoring,saturated-fat_100g_scoring,fiber_100g_scoring,proteins_100g_scoring,bio_scoring
0,0000000003087,1474103893,Farine de blé noir,"en:to-be-completed,en:nutrition-facts-to-be-co...",NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000024600,1434535914,Filet de bœuf,"en:to-be-completed,en:nutrition-facts-to-be-co...",Filet-de-boeuf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,en:france,NaN,NaN,NaN,http://fr.openfoodfacts.org/images/products/00...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000000027205,1458238638,NaN,"en:to-be-completed,en:nutrition-facts-to-be-co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,NaN,NaN,NaN,http://fr.openfoodfacts.org/images/products/00...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000036252,1489055667,Lion Peanut x2,"en:to-be-completed,en:nutrition-facts-complete...",NaN,Sunridge,sunridge,e,1883.0,57.5,0.09652,12.5,2.5,2.5,0.0,unknown,unknown,NaN,"en:france,en:united-states",en:e322,1.0,"Milk chocolate coating (evaporated cane juice,...",http://fr.openfoodfacts.org/images/products/00...,1.0,10.0,2.0,5.0,1.0,5.0,1.0,1.0,2.0,1.0,NaN
4,0000000039259,1473538082,Twix x2,"en:to-be-completed,en:nutrition-facts-to-be-co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,en:france,NaN,NaN,NaN,http://fr.openfoodfacts.org/images/products/00...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
